**IMPORTS**

In [2]:
import numpy as np
from PIL import Image, ImageOps
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from matplotlib.pyplot import imshow
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img, save_img
from sklearn.model_selection import KFold

**CONSTANTS**

In [3]:
pics_waldo_dir = 'Hey-Waldo\\64-waldo'
pics_notwaldo_dir = 'Hey-Waldo\\64-notwaldo'

**LOADING ORIGINAL WALDO'S PICS**

In [3]:
x_waldo = list()
for filename in os.listdir(pics_waldo_dir):
    x_waldo_temp = img_to_array(load_img(pics_waldo_dir+'\\'+filename))
    x_waldo.append(x_waldo_temp.reshape((1,) + x_waldo_temp.shape))

**LOADING ALL WALDO'S PICS**

In [4]:
x_waldo = list()
for filename in os.listdir(pics_waldo_dir):
    x_waldo.append(np.asarray(Image.open(pics_waldo_dir+'\\'+filename)))
y_waldo = np.ones(len(x_waldo))

**LOADING NOT WALDO'S PICS**

In [5]:
x_notwaldo = list()
for filename in os.listdir(pics_notwaldo_dir):
    aux_img = Image.open(pics_notwaldo_dir+'\\'+filename)
    aux_img = aux_img.convert('RGB')
    x_notwaldo.append(np.asarray(aux_img))
y_notwaldo = np.zeros(len(x_notwaldo))

**CREATING DATASET**

In [6]:
X = np.array(x_waldo + x_notwaldo)
Y = np.array(list(y_waldo) + list(y_notwaldo))[:int(len(Y)/10)]
X = X.reshape(len(X), 64, 64, 3)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
#for i in range(len(y_train)):
#    if y_train[i] == 1:
#        imshow(X_train[i])
#        break
#y_train = to_categorical(y_train, num_classes = 2)
#y_test = to_categorical(y_test, num_classes = 2)

**CREATING CNN**

In [8]:
#model = Sequential()
#model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(64,64,3)))
#model.add(Conv2D(32, kernel_size=3, activation='relu'))
#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])
    
    cnn = Sequential()
    cnn.add(Conv2D(filters=32, kernel_size=(2,2), strides=(1,1),padding='same',input_shape=(64,64,3),data_format='channels_last'))
    cnn.add(Activation('relu'))
    cnn.add(MaxPooling2D(pool_size=(2,2),strides=2))
    cnn.add(Conv2D(filters=64,kernel_size=(2,2),strides=(1,1),padding='valid'))
    cnn.add(Activation('relu'))
    cnn.add(MaxPooling2D(pool_size=(2,2),strides=2))
    cnn.add(Flatten())        
    cnn.add(Dense(64))
    cnn.add(Activation('relu'))
    cnn.add(Dropout(0.25))
    cnn.add(Dense(1))
    cnn.add(Activation('sigmoid'))
    cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])
    
    cnn.fit(x_train, y_train, epochs=3)
    
    print('Model evaluation ',cnn.evaluate(x_test,y_test))

Epoch 1/3
744/744 [==============================] - 35s 47ms/step - loss: 0.0179 - auc: 0.0000e+00
Epoch 2/3
744/744 [==============================] - 34s 46ms/step - loss: 0.0000e+00 - auc: 0.0000e+00
Epoch 3/3
186/186 [==============================] - 2s 13ms/step - loss: 1536.9050 - auc: 0.5000
Model evaluation  [1536.905029296875, 0.5]
Epoch 1/3
744/744 [==============================] - 34s 45ms/step - loss: 1.0566 - auc: 0.8060
Epoch 2/3
744/744 [==============================] - 34s 45ms/step - loss: 0.1389 - auc: 0.9367
Epoch 3/3
186/186 [==============================] - 2s 13ms/step - loss: 0.0418 - auc: 0.0000e+00: 0s - loss: 0.0413 - auc: 0.0000e+0
Model evaluation  [0.041750382632017136, 0.0]
Epoch 1/3
744/744 [==============================] - 34s 46ms/step - loss: 1.0774 - auc: 0.8208
Epoch 2/3
744/744 [==============================] - 34s 46ms/step - loss: 0.1424 - auc: 0.9316
Epoch 3/3
186/186 [==============================] - 2s 12ms/step - loss: 0.0196 - auc: 0.

**TRAINING**

In [12]:
cnn.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=6)

Epoch 1/6
744/744 [==============================] - 36s 48ms/step - loss: 0.7968 - auc: 0.7949 - val_loss: 0.1724 - val_auc: 0.9173
Epoch 2/6
744/744 [==============================] - 35s 47ms/step - loss: 0.1145 - auc: 0.9363 - val_loss: 0.0960 - val_auc: 0.9670
Epoch 3/6
744/744 [==============================] - 36s 48ms/step - loss: 0.0823 - auc: 0.9651 - val_loss: 0.0749 - val_auc: 0.9816
Epoch 4/6
744/744 [==============================] - 35s 47ms/step - loss: 0.0662 - auc: 0.9758 - val_loss: 0.0813 - val_auc: 0.9725
Epoch 5/6
744/744 [==============================] - 35s 48ms/step - loss: 0.0497 - auc: 0.9865 - val_loss: 0.0958 - val_auc: 0.9641
Epoch 6/6
744/744 [==============================] - 36s 48ms/step - loss: 0.0399 - auc: 0.9913 - val_loss: 0.0547 - val_auc: 0.9867


In [9]:
def crop(path, input, height, width):
    im = Image.open(input)
    imgwidth, imgheight = im.size
    for i in range(0,imgheight,int(height/2)):
        for j in range(0,imgwidth,int(width/2)):
            box = (j, i, j+width, i+height)
            a = im.crop(box)
            try:
                o = a.crop((0,0,height, width))
                pred_arr = np.asarray(o).reshape(1,64,64,3)
                pred = cnn.predict(pred_arr)
                if pred[0] > 0.9:
                    print(pred)
                    #a.show()
                
            except:
                pass

In [10]:
crop('C:\\Users\\Bruno\\Downloads\\Projetos\\AIFindsWaldo', '2.jpg', 64, 64)

In [11]:
print(cnn.predict(np.asarray(Image.open('3_8_10.jpg')).reshape(1,64,64,3)))
print(cnn.predict(np.asarray(Image.open('3_8_11.jpg')).reshape(1,64,64,3)))
print(cnn.predict(np.asarray(Image.open('3_8_12.jpg')).reshape(1,64,64,3)))
print(cnn.predict(np.asarray(Image.open('20-3.jpg')).reshape(1,64,64,3)))

[[0.13913803]]
[[0.02654611]]
[[1.6730268e-05]]
[[0.21566589]]
